# Introduction

This document illustrates how to build the recommender system with Mapreduce. The original Netflix dataset is around 2G and in this project we only take 2M data from the original data which is preprocessed into the form of user, movie, rating. The preprocess method is beyond the scope of this project.

# Theoretical Background

There are generally two basic algorithms for the recommender system, user collaborative filtering (user CF) and item collaborative filtering (item CF). User CF is a form of collaborative filtering based on the similarity between users calculated using people's ratings of those items
while item CF is based on the similarity between items.

In this project, we user item CF instead of user CF for the following reasons:

* The number of users weighs more than the number of products, which makes item CF more computationally efficient.
* Items (movie) will not change frequently in this project, while users’ preferences may change frequently. Item CF can help save computation.
* Recommending movies based on users’ historical data makes more sense intuitively. 

To build the item CF model we first need to define the relationship between items. Some possible strategies are making use of the categories, producers, actors, countries, years, earnings and so on. Here, however,  we use rating history to build relationships between movies based on limited data, which means the number of users rated two movies denotes the level of how these two movies are connected.

Next I am gonna introduce the concept of co-occurrence matrix. A co-occurrence matrix is a matrix that is defined over an image to be the distribution of co-occurring pixel values (grayscale values, or colors) at a given offset. In this project co-occurrence matrix denotes the frequency any two movies show up together. Then we normalize the co-occurrence matrix and multiply it with the user rating matrix to get the final result, as shown in the figure below. 

![](https://drive.google.com/file/d/1sriXLf1x7wZn9Je36hjc-AAu87I0bmV4/view?usp=sharing)
https://drive.google.com/file/d/1sriXLf1x7wZn9Je36hjc-AAu87I0bmV4/view?usp=sharing

Note that for missing values in the user rating matrix we use the average rating of that user to fill in the vacancy. 


# Algorithm

The core of the algorithm is matrix multiplication in Mapreduce. This section lists all seven classes of this project in sequence and their functionalities.
 
## movieListCollect
 
Read in raw data and form the list of all movies for later use. 
 
**Mapper**
* input value: input user,movie,rating
* output key: movie
* output value: empty string

**Reducer**
* input key: movie
* input value: ""
* output key: movie
* output value: ""
 

## setupRating
 
Read in raw data and fill all missing ratings with the average rating of the user. Output the movie rating information of the user.
 
**Mapper**
* input value: user,movie,rating
* output key: user
* output value: movie:rating
 
**Reducer**
* input key: user
* input value: movie:rating
* output key: user
* output value: movie,rating


## dataDividerByUser
 
Read in raw data and output the movie rating information of the user without filling in missing ratings.
 
**Mapper**
* input: user,movie,rating
* output key: user
* output value: movie:rating
 
**Reducer**
* input key = userID
* input value = <movie1:rating1, movie2:rating2, ......>
* output key = userID
* outputValue = movie1:rating1,movie2:rating2
*  

## coOccurrenceMatrixGenerator

Iterate through the output of dataDividerByUser and build the occurrence matrix based on the results from dataDividerByUser. 
 
**Mapper**
* input value = userid\tmovie1:rating,movie2:rating...
* output key: movieA:movieB
* output value: 1
 
**Reducer**
* input key: movieA:movieB
* input value: <1,1,1,1, ......>
* output key: movieA:movieB
* output value: sum (= 1+1+1 ......)
 

## Normalize
 
Normalize the co-occurrence matrix by row based on the results from coOccurrenceMatrixGenerator.
 
**Mapper**
* input value: movieA:movieB\t relation movie1:movie2\t432
* output key: movieA
* output value: movieB=relation
 
**Reducer**
input key: movieA
input value: <movieB1=relation1, movieB2=relation2, ......>
output key: movieB
output value: movieA=relative_relation
 

## multiplication
 
Assume (movieA, movieB) is an element in co-occurrence matrix and (movie, user) is an element in user rating matrix. Two mappers collect elements from left and right matrices and record all the cross products (for multiplication, column of left matrix must match row from right column). Output the sub- ratings. Note that the normalized occurrence matrix is from Normalize class while user rating matrix is from setupRating class (with missing rating filled).
 
**Mapper**
* input value: movieB \t movieA=relation
* output key: movieB
* output value: movieA=relation
 
**Mapper**
* input value: user,movie,rating
* output key: movie
* output value: user:rating (try avoid using "=")
 
**Reducer**
* input key: movieB
* input value: <movie1=relation, movie2=relation, user1:rating, user2:rating...>
* output key: movieID:userID
* output value: sub_rating
 

## sum
 
Read data from multiplication and sum up all the sub-ratings that from the same user and same movie.
 
**Mapper**
* input value: user:movieA \t sub_rating
* output key: user:movieA
* output value: sub_rating
 
**Reducer**
* input key: user:movieA
* input value: <subSum, subSum, ......>
* output key: user:movieA
* output value:

# Implementation

This section records the procedure of implementing the algorithm in the Mac system. Since Mapreduce requires a Linux system, we use Docker containers to build the environment, which consists of one master node and two slave nodes.

First we need to get into the Linux environment by docker and start hadoop.

*./start-container.sh # start docker container   
./start-hadoop.sh # start hadoop*

Now we should be able to see root@hadoop-master:~#. 

*cd src # enter src file

*cd RecommenderSystem # this is the file path where we put the code

*hdfs dfs -mkdir /input # make a new directory*

*hdfs dfs -put input/* /input  # upload raw data, which is in txt format***

Remove all the output file path if built previously

*hdfs dfs -rm -r /dataDividedByUser
hdfs dfs -rm -r /coOccurrenceMatrix
hdfs dfs -rm -r /Normalize
hdfs dfs -rm -r /Multiplication
hdfs dfs -rm -r /Sum
hdfs dfs -rm -r /movieList 
hdfs dfs -rm -r /newRating*

*cd src/main/java/
hadoop com.sun.tools.javac.Main *.java
jar cf recommender.jar *.class*

Run the code

*hadoop jar recommender.jar Driver /input /dataDividedByUser /coOccurrenceMatrix /Normalize /Multiplication /Sum /movieList /newRating

*hdfs dfs -cat /Sum/**

*hdfs dfs -ls / # check the final outputs**

# Conclusion

This document focuses more on the algorithms and general steps to implement the recommender system. For more information on Docker, or Mapreduce, or Linux system, please go to the official websites or jiuzhang.com. 

When you run the algorithm, we suggest you first run it with very simple test data. Just for your reference, 2M of data takes around 40 minutes. To evaluate the results, we can simply compare the calculated ratings of the user with its original ratings. 

# Acknowledgement

This project is the course project from one of the jiuzhang.com online courses. Please refer to jiuzhang.com for details.